In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from torch import nn, optim

In [44]:
data = 'D:\장우영\LOCALSEARCH\DA\DA\data\FAmerge_20230518_124958.csv'

merge_data = pd.read_csv(data, encoding='ANSI')
#print(weather_data)
print(merge_data.head(1))

        mmsi ship_name  ship_type   
0  440051540      D-01          0  \

                                                geom    cog  sog   
0  0101000020110F00000000002039676B41000000002DCF...  329.2  5.7  \

        insert_time  지방청           표지                일시   풍향   유향    기온    수온   
0  2023-05-11 10:10  부산청  신항유도등부표(랜비)  2023-05-11 10:10  196  246  16.4  17.0  \

     풍속   유속    기압  습도  
0  9.77  0.3  1017  82  


In [40]:
# 데이터를 교육 및 테스트 세트로 분할
X = merge_data[["mmsi","ship_name","ship_type","cog","sog","insert_time","풍향","기온","수온","풍속","유속","기압","습도"]]  # 입력 값 (excluding "geom")
y = merge_data["geom"]  # 출력 값 (geom)

print(X)
#print(y)

            mmsi      ship_name  ship_type    cog   sog       insert_time   
0      440051540           D-01          0  329.2   5.7  2023-05-11 10:10  \
1      440300780            NaN          0  329.8   0.0  2023-05-11 10:10   
2      440208550            NaN          0  139.3   0.0  2023-05-11 10:10   
3      440275000  GBK EXPRESS 1         40  175.8   0.0  2023-05-11 10:10   
4      440414850   NO1 GEO SUNG         80    0.0   0.0  2023-05-11 10:10   
...          ...            ...        ...    ...   ...               ...   
16180  538010219  SAWASDEE VEGA         70  225.2  16.0  2023-05-11 10:29   
16181  440121850            NaN          0    0.0   0.0  2023-05-11 10:30   
16182  440110850  PILOT ORYUKDO         50  322.3   1.7  2023-05-11 10:30   
16183  440107840   NO.7 GEUMHWA         30    0.0   0.0  2023-05-11 10:30   
16184  440134620       HYUNJUNG         80  231.3   0.7  2023-05-11 10:30   

        풍향    기온    수온     풍속   유속    기압  습도  
0      196  16.4  17.0   9.7

In [42]:
print(merge_data["ship_name"])

0                 D-01
1                  NaN
2                  NaN
3        GBK EXPRESS 1
4         NO1 GEO SUNG
             ...      
16180    SAWASDEE VEGA
16181              NaN
16182    PILOT ORYUKDO
16183     NO.7 GEUMHWA
16184         HYUNJUNG
Name: ship_name, Length: 16185, dtype: object


In [36]:
#학습용, 검증용, 시험용으로 분리
train_size = int(327 * 0.8)
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+33], y[train_size:train_size+33]
X_test, y_test = X[train_size+33:], y[train_size+33:]

In [37]:
import torch
#Tensor 형태로 변환
import numpy as np

def make_Tensor(array):
    return torch.from_numpy(array).float()

X_train = make_Tensor(X_train.to_numpy())
y_train = make_Tensor(y_train.to_numpy())
X_val = make_Tensor(X_val.to_numpy())
y_val = make_Tensor(y_val.to_numpy())
X_test = make_Tensor(X_test.to_numpy())
y_test = make_Tensor(y_test.to_numpy())

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_268\3470333947.py", line 8, in <module>
    X_train = make_Tensor(X_train.to_numpy())
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_268\3470333947.py", line 6, in make_Tensor
    return torch.from_numpy(array).float()
TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 2102, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\SW\AppData\Roaming\Python\Pyth

In [47]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 데이터 전처리
# merge_data DataFrame을 사용하여 필요한 전처리를 수행합니다.
# 필요한 전처리에는 NaN 값 처리, 범주형 변수 인코딩, 수치형 변수 스케일링 등이 포함될 수 있습니다.

# 학습용, 검증용, 시험용으로 분할
train_size = int(len(merge_data) * 0.8)
val_size = int(len(merge_data) * 0.1)
test_size = len(merge_data) - train_size - val_size

X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1])))
model.add(Dense(2))  # 2개의 출력 뉴런: geom
model.compile(loss='mean_squared_error', optimizer='adam')

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val))

# 모델 평가
train_loss = model.evaluate(X_train, y_train)
val_loss = model.evaluate(X_val, y_val)
test_loss = model.evaluate(X_test, y_test)
print(f"Train Loss: {train_loss}, Val Loss: {val_loss}, Test Loss: {test_loss}")

# 예측
next_location = model.predict(X_test[-1].reshape(1, X_test.shape[1], X_test.shape[2]))
print(f"Next location prediction (geom): {next_location}")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\SW\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\SW\AppData\Local\Temp\ipykernel_268\3973625044.py", line 22, in <module>
    model.add(LSTM(128, input_shape=(X_train.shape[1])))
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\layers\rnn\lstm.py", line 561, in __init__
    super().__init__(
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\layers\rnn\dropout_rnn_cell_mixin.py", line 43, in __init__
    super().__init__(*args, **kwargs)
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\layers\rnn\base_rnn.py", line 271, in __init__
    super().__init__(**kwargs)
  File "c:\Users\SW\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\trackable\base.py", line 205, in _method_wrapper
 

c
12
